In [1]:
import tensorflow as tf
import yaml
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers, models

train_dir = 'C:/Users/ouaha/OneDrive/Bureau/projet app stat/archive/data/Training'  # Remplacez par le chemin réel du dossier d'entraînement
test_dir = "C:/Users/ouaha/OneDrive/Bureau/projet app stat/archive/data/Testing"         # Remplacez par le chemin réel du dossier de test
with open('C:/Users/ouaha/OneDrive/Bureau/projet app stat/archive/config.yml', 'r') as file:
    params = yaml.safe_load(file)

preprocessing_params = params['preprocessing']
target_size = tuple(preprocessing_params['target_size'])
normalize = preprocessing_params['normalize']
standardize = preprocessing_params['standardize']
grayscale = preprocessing_params['grayscale']

In [2]:
def load_and_preprocess_image(path):
    # Charger l'image
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=1 if grayscale else 3)  # Convertir en niveaux de gris si spécifié
    img = tf.image.resize(img, target_size)  # Redimensionner l'image

    if normalize:
        img = img / 255.0  # Normaliser entre 0 et 1

    if standardize:
        img = (img - tf.reduce_mean(img)) / tf.math.reduce_std(img)  # Standardisation

    return img

def prepare_dataset(directory):
    images = []
    labels = []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                images.append(load_and_preprocess_image(img_path))
                labels.append(label)  # ou utiliser un encodage d'étiquettes

    return tf.data.Dataset.from_tensor_slices((images, labels))

train_dataset = prepare_dataset(train_dir)
test_dataset = prepare_dataset(test_dir)

In [3]:
full_dataset = train_dataset.shuffle(buffer_size=2000)

# Définir le pourcentage pour la validation
validation_split = 0.2  # 10% pour la validation
num_samples = tf.data.experimental.cardinality(full_dataset).numpy()  # Nombre total d'échantillons
num_validation = int(num_samples * validation_split)  # Nombre d'échantillons pour la validation

# Créer les ensembles d'entraînement et de validation
val_dataset = full_dataset.take(num_validation)  # Prendre les premiers num_validation échantillons
train_dataset = full_dataset.skip(num_validation)  # Sauter ces échantillons pour l'ensemble d'entraînement

print(f'Taille de l\'ensemble d\'entraînement : {tf.data.experimental.cardinality(train_dataset).numpy()}')
print(f'Taille de l\'ensemble de validation : {tf.data.experimental.cardinality(val_dataset).numpy()}')

Taille de l'ensemble d'entraînement : 2296
Taille de l'ensemble de validation : 574


In [4]:
labels_seen = set()

for image, label in val_dataset:
    label_decoded = label.numpy().decode('utf-8')
    labels_seen.add(label_decoded)

print("Labels présents dans l'ensemble de validation :", labels_seen)

Labels présents dans l'ensemble de validation : {'pituitary_tumor', 'meningioma_tumor', 'glioma_tumor', 'no_tumor'}


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Input(shape=(256, 256, 3)),

    # Bloc 1
    layers.Conv2D(32, (3, 3), activation='relu'),

    # Bloc 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Bloc 3
    layers.Conv2D(128, (3, 3), activation='relu'),

    # Bloc 4
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Couche Fully Connected
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),  # Pour éviter le surapprentissage
    layers.Dense(4, activation='softmax')

])


# Création du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [6]:
label_map = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['glioma_tumor', 'pituitary_tumor', 'no_tumor', 'meningioma_tumor']),
        values=tf.constant([0, 1, 2, 3]),
    ),
    default_value=tf.constant(-1)  # Default value if key is missing
)

# Function to map labels using the lookup table
def map_labels(x, y):
    return x, label_map.lookup(y)

# Apply the mapping to the dataset
train_dataset = train_dataset.map(map_labels)
val_dataset = val_dataset.map(map_labels)

In [7]:
train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)


history=model.fit(train_dataset, epochs=10, validation_data= val_dataset)

Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1007s 14s/step - accuracy: 0.4954 - loss: 3.7429 - val_accuracy: 0.6760 - val_loss: 0.7238
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1002s 14s/step - accuracy: 0.6949 - loss: 0.6871 - val_accuracy: 0.7909 - val_loss: 0.5055
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1000s 14s/step - accuracy: 0.8267 - loss: 0.4490 - val_accuracy: 0.8920 - val_loss: 0.2914
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1033s 14s/step - accuracy: 0.8564 - loss: 0.3228 - val_accuracy: 0.9460 - val_loss: 0.1743
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1067s 15s/step - accuracy: 0.9049 - loss: 0.2303 - val_accuracy: 0.9495 - val_loss: 0.1642
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1029s 14s/step - accuracy: 0.9218 - loss: 0.1926 - val_accuracy: 0.9808 - val_loss: 0.0730
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1049s 14s/step - accuracy: 0.9384 - loss: 0.1516 - val_accuracy: 0.9756 - val_loss: 0.0938
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 1075s 15s/step - accuracy: 0.9472 - loss: 0.1459 - val_accu

In [9]:

model_save_path = 'C:/Users/ouaha/OneDrive/Bureau/projet app stat/archive/models/my_trained_model.keras'
model.save(model_save_path)
print(f"Modèle sauvegardé dans {model_save_path}")


Modèle sauvegardé dans C:/Users/ouaha/OneDrive/Bureau/projet app stat/archive/models/my_trained_model.keras
